**Use PowerShell kernel to run code in this notebook**

This notebook provides code that creates Logic App and related required artifacts.

Assumption is that following prerequisits are already in place:

- Azure Subscription
- Azure Storage Account
- Azure CLI (w/ Bicep extension)

First open PowerShell, set some variables and login to Azure.

Note: Update variables below fit specific needs

In [ ]:

$azure_subscription = '<Subscription ID>'
$azure_region = '<Location>'
$azure_resource_group = '<Resource group>'

$logic_app_bicep_template = 'Bicep\Daily.bicep'
$logic_app_name = '<Logic App name>'

$api_connection_name_suffix = '<suffix>'

$azure_storage_account = '<Storage account name>'
$azure_storage_account_folder_path = '<Storage account folder path>'
$azure_storage_account_resource_group = '<Storage account resource group>'

az login

Run following command, in case some az extensions are missing and need to be installed. Installation will happen automatically and without prompt

In [ ]:
az config set extension.use_dynamic_install=yes_without_prompt

Create Resource Group that will hold required Azure resources. Next cell can be skipped in case an existing Resource Group will be used

In [ ]:
az group create     -l $azure_region `
                    -n $azure_resource_group

Logic App with all code already is saved as Bicep template. Following cell will create Logic App

In [ ]:
az deployment group create  --resource-group $azure_resource_group `
                            --template-file $logic_app_bicep_template `
                            --parameters    azure_subscription=$azure_subscription `
                                            api_connection_name_suffix=$api_connection_name_suffix `
                                            logic_app_name=$logic_app_name `
                                            azure_storage_account=$azure_storage_account `
                                            azure_storage_account_folder_path=$azure_storage_account_folder_path

Logic App is created as Disabled. Before it can be run, it needs to be enabled. Code in cell below _should_ do this, but it appears there is a bug - reported here - ["az logic workflow update" removes API connection · Issue #20807 · Azure/azure-cli (github.com)](https://github.com/Azure/azure-cli/issues/20807)

Until issue is resolved, Logic App can be enabled in Azure Portal.

In [ ]:
#az logic workflow update --name $logic_app_name `
#                         --resource-group $azure_resource_group `
#                         --state Enabled

Logic App needs to call Azure Consumption REST API and for that requires Reader RBAC at subscription level.

First, get Logic App managed identity

In [ ]:
$logic_app_id = az resource show    --name $logic_app_name `
                                    --resource-group $azure_resource_group `
                                    --resource-type Microsoft.Logic/workflows `
                                    --query identity.principalId | ConvertFrom-Json

Now grant Logic App managed identity Reader RBAC at subscription level

In [ ]:
az role assignment create   --assignee $logic_app_id `
                            --role Reader `
                            --subscription $azure_subscription

Logic App needs Storage Blob Data Contributor RBAC on storage account where data will be stored

In [ ]:
az role assignment create   --assignee $logic_app_id `
                            --role 'Storage Blob Data Contributor' `
                            --scope "/subscriptions/$azure_subscription/resourceGroups/$azure_storage_account_resource_group/providers/Microsoft.Storage/storageAccounts/$azure_storage_account"

At this point, go to Azure Portal and execute Logic App. All being well, Logic App should be able to call Azure Consumption REST API and create CSV file in storage account.

Note: It can take a few minutes for RBAC's to become effective. If Logic App returns 403 error when trying to make REST API call or write data to storage account, wait for another few minutes and try again.